## Introduction

This notebook is about setting up and running Minecraft. Why? That doesn't sound like mathematics...

Well first of all Minecraft represents a very precise mathematics-based world. (Not too much physics 
but lots of regular geometry.) Second we can set up some software that talks to the Minecraft world
and creates new things. And thirdly what we choose to create can be as mathematical as we like! 

We will write our code in Python which in turn speaks with the Minecraft server... so first let's 
look into the necessary steps.

1. Purchase or check out from the library the book _Learn to Program With Minecraft_ by Craig Richardson
(ISBN 9781593276706).
2. Go through Chapter 1 on setting up Minecraft and Python (for your type of computer)
3. Start Minecraft. You have to purchase it (about $20) and log in when it starts. Play for a bit!
  - This starts as an application
4. Install Python per the instructions. There are two popular versions: 2.7 and 3.5. We will use 3.5.
5. Install Java per the instructions (we're almost there!)
6. Install the Minecraft Python API and the Minecraft Server (called Spigot)
7. Run Spigot as instructed... that is the Server running, waiting for players to show up.
  - This launches from a sub-folder of Python Minecraft in my Documents folder
8. If necessary: Re-start Minecraft. Click on Multiplayer. Click Add Server, that should launch the game.
9. Notice you have the option of configuring everything to run offline. That is a good option to have.
10. Start the IDLE editor and get going with the basics of creating a Python program that talks to Minecraft.

Notice that in the program blocks given below there is the following line of code: 

```
from mcpi.minecraft import Minecraft
```

This is the important/magical bit: It connects the program to a library, and in the Python language 
the library is the powerful thing that allows you to do amazing things with simple commands. 

### What this code does

This code is a kind of factory. In Minecraft it will build structures: 

Enormous hollow spheres. 

Triangular plates.

Pathways in the sky.

Castles in the sky that leak water and lava in fountains to the ground.


In [1]:
# save and F5 to run

from mcpi.minecraft import Minecraft
import numpy as np
import random as r
import time

mc = Minecraft.create()

# block types:
#   0      air
# 155      quartz
# 152      redstone
#  12      sand

quartz = 155
air = 0

def Shell(mc,pos,radius,blockType):
    minrad = radius - 1
    maxrad = radius
    xc = pos.x
    yc = pos.y
    zc = pos.z
    for x in range(int(xc-radius-1), int(xc+radius+2)):
        for y in range(int(yc-radius-1), int(yc+radius+2)):
            for z in range(int(zc-radius-1), int(zc+radius+2)):
                distance = np.sqrt((x-xc)*(x-xc)+(y-yc)*(y-yc)+(z-zc)*(z-zc))
                if distance > minrad and distance < maxrad:
                    mc.setBlock(x,y,z,blockType)

# create a block bar between a and b (sparse)
def Bar(mc,a,b,blockType):
    if a == b:
        mc.setBlock(a.x, a.y, a.z, blockType)
        return
    dx = np.abs(a.x-b.x)
    dy = np.abs(a.y-b.y)
    dz = np.abs(a.z-b.z)
    if dx >= dy and dx >= dz:
        if a.x > b.x:
            c = b
            b = a
            a = c
        my = (b.y - a.y) / (b.x - a.x)
        ny = a.y
        mz = (b.z - a.z) / (b.x - a.x)
        nz = a.z
        for xloc in range(int(a.x),int(b.x)+1):
            x = 1.0*xloc
            xrel = x - a.x
            y = my * xrel + ny
            z = mz * xrel + nz
            mc.setBlock(x,y,z,blockType)
    elif dy >= dx and dy >= dz:
        if a.y > b.y:
            c = b
            b = a
            a = c
        mx = (b.x - a.x) / (b.y - a.y)
        nx = a.x
        mz = (b.z - a.z) / (b.y - a.y)
        nz = a.z
        for yloc in range(int(a.y),int(b.y)+1):
            y = 1.0*yloc
            yrel = y - a.y
            x = mx * yrel + nx
            z = mz * yrel + nz
            mc.setBlock(x,y,z,blockType)
    else:
        if a.z > b.z:
            c = b
            b = a
            a = c
        mx = (b.x - a.x) / (b.z - a.z)
        nx = a.x
        my = (b.y - a.y) / (b.z - a.z)
        ny = a.y
        for zloc in range(int(a.z),int(b.z)+1):
            z = 1.0*zloc
            zrel = z - a.z
            x = mx * zrel + nx
            y = my * zrel + ny
            mc.setBlock(x,y,z,blockType)

def Plate(mc, a, b, c, blockType):
    tv = b - a
    mag = np.sqrt(tv.x*tv.x + tv.y*tv.y + tv.z*tv.z)
    tvn = tv
    tvn.x /= mag
    tvn.y /= mag
    tvn.z /= mag
    ab = a
    for i in range(int(mag)+1):
        Bar(mc, ab, c, quartz)
        ab += tvn
    mc.postToChat("made a plate")
                  
def RandomNearby(mc, scale):
    pos = mc.player.getPos()
    pos.x += scale + 2.*scale*r.random()
    pos.y += 2.*scale*r.random()
    pos.z += -2.*scale + 4.*scale*r.random()
    return pos
    
##############
##
## Program: Plates
##
##   Creates a set of triangular plates
##
##############
##for i in range(3):
##    Plate(mc, RandomNearby(mc, 40), RandomNearby(mc, 40), RandomNearby(mc, 40), quartz)


    
## Clock to check latency on simple setBlock rendering
# p = RandomNearby(mc, 10)  # p is the position of the binary block; this will toggle
p = mc.player.getPos()
state = True

for i in range(20):
    for j in range(20):
        for k in range(20):

            mc.setBlock(p.x+i, p.y+j, p.z+k+10, r.randint(0,255))
            
            if state:
                mc.setBlock(p.x, p.y, p.z, quartz)
                state = False
                print('quartz')
                # time.sleep(1)
            else:
                mc.setBlock(p.x, p.y, p.z, air)
                state = True
                print('air')
                # time.sleep(1)
                

##############
##
## Program: 3D Graph
##
##   Creates a set of small spheres connected with bars
##
##############
# firstTime = True
# lastPos = mc.player.getPos()
# bagScale = 60.
##for i in range(30):
##    pos = mc.player.getPos()
##    pos.x += bagScale + 2.*bagScale*r.random()
##    pos.y += 2.*bagScale*r.random()
##    pos.z += -2.*bagScale + 4.*bagScale*r.random()
##    Shell(mc, pos, 4, 138)
##    if firstTime:
##        firstTime = False
##    else:
##        Bar(mc,lastPos,pos,155)
##        lastPos = pos





ImportError: No module named mcpi.minecraft